In [1]:
import cv2
import numpy as np
import maxflow
import graphmaker
from icecream import ic
import tomasi

In [2]:
L = cv2.imread("/kaggle/input/mystereo/scene_L.ppm")
R = cv2.imread("/kaggle/input/mystereo/scene_R.ppm")
L = cv2.resize(L, dsize=None, fx=0.5, fy=0.5)
R = cv2.resize(R, dsize=None, fx=0.5, fy=0.5)


In [3]:
dispMin=-8
dispMax=8
dispSize=dispMax-dispMin+1

maxIter=1


occ=2**31-1
K=-1
lambda1=-1
lambda2=-1
denom=1

In [4]:
currEnergy=0
nbs = [(0, -1), (1, 0)]
max_denom=16
edgethresh=8
disparity_map=np.ones_like(np.zeros((L.shape[0], L.shape[1]), dtype=np.int64), dtype=np.int64)*occ
vars_pre = np.zeros((L.shape[0], L.shape[1]), dtype=np.int64)
vars_post = np.zeros((L.shape[0], L.shape[1]), dtype=np.int64)
iterator = np.ones(shape=(L.shape[0], L.shape[1]))

# print(disparity_map.shape, vars_pre.shape, vars_post.shape, iterator.shape)

(288, 384) (288, 384) (288, 384) (288, 384)


In [5]:

def setParams():
	global K,denom, lambda1, lambda2, tom
	k = int((dispSize+2) / 4)
	if k < 3:
		k = 3

	sum_val = 0
	num = 0
	xmin = - dispMin
	xmax = R.shape[1] - dispMax

	y_range = np.arange(L.shape[0])
	x_range = np.arange(xmin, xmax)
	d_range = np.arange(dispMin, dispMax + 1)

	for y in y_range:
		for x in x_range:
			delta = data_penalty_color((y, x), (y, x + d_range))
			delta.sort()
			sum_val += delta[k-1]
			num += 1
	assert (num != 0 and sum_val != 0)

	K = sum_val / num
	lambda2=K/5
	lambda1=3*lambda2

	print("K obtained:", K)

	min_error=float(2**30)
	for i in range(1,max_denom+1):
		tempK=int(i*K+0.5)
		tempLambda1=int(i*lambda1+0.5)
		tempLambda2=int(i*lambda2+0.5)
		e=abs(tempK/(i*K)-1.0)+abs(tempLambda1/(i*lambda1)-1.0)+abs(tempLambda2/(i*lambda2)-1.0)
		if e<min_error:
			min_error=e
			K=tempK
			lambda1=tempLambda1
			lambda2=tempLambda2
			denom=i

def data_penalty_color(yx1, yx2):
	global L,R,tom
	y1, x1 = yx1
	y2, x2 = yx2
	return tom.get_tomasi_penalty(yx1,yx2,L,R, cutoff=30)
	# return np.sum(np.abs(L[y1, x1] - R[y2, x2])**2, axis=-1)/3


In [6]:
def data_occ_penalty(yx1, yx2):
    return data_penalty_color(yx1, yx2)*denom + K

def smooth_penalty(yx1, yx2, d):
    dMax=0
    for i in range(3):
        dMax=max(dMax,abs(L[yx1[0],yx1[1],i]-L[yx2[0],yx2[1],i]))
        dMax=max(dMax,abs(R[yx1[0],yx1[1]+d,i]-R[yx2[0],yx2[1]+d,i]))
    return lambda1 if dMax<=edgethresh else lambda2

def in_bounds(yx):
    return 0 <= yx[0] < L.shape[0] and 0 <= yx[1] < L.shape[1]

def findEnergy():
    energy=0
    for i,_ in np.ndenumerate(iterator):
        d1=disparity_map[i]
        if d1!=occ:
            energy+=data_occ_penalty(i,(i[0],i[1]+d1))
        for n in nbs:
            i2=(i[0]+n[0],i[1]+n[1])
            if in_bounds(i2):
                d2 = disparity_map[i2]
                if d1.all()==d2.all():
                    continue
                if d1!=occ and in_bounds((i2[0],i2[1]+d1)):
                    energy+=smooth_penalty(i,i2,d1)
                if d2!=occ and in_bounds((i[0],i[1]+d2)):
                    energy+=smooth_penalty(i,i2,d2)
    return energy

In [7]:
def buildNodes(gm,p,alpha):
  
    global disparity_map, vars_pre, vars_post
    d=disparity_map[p]
    q=(p[0],p[1]+d)

    #print(p,q)
    if alpha==d:
        vars_pre[p]=-1
        vars_post[p]=-1
        gm.grEnergy+=data_occ_penalty(p,q)
        return
    if d!=occ:
        vars_pre[p]=gm.add_binvar(data_occ_penalty(p,q),0)
        pass
    else:
        vars_pre[p]=-2

    q=(p[0],p[1]+alpha)
    if in_bounds(q):    
        vars_post[p]=gm.add_binvar(0,data_occ_penalty(p,q))
    else:
        vars_post[p]=-2


def buildSmooth(gm,p,q,alpha):
    d1=disparity_map[p]
    d2=disparity_map[q]
    pre1=vars_pre[p]
    pre2=vars_pre[q]
    post1=vars_post[p]
    post2=vars_post[q]

    if post1!=-2 and post2!=-2:
        delta=smooth_penalty(p,q,alpha)
        if post1!=-1:
            if post2!=-1:
                gm.energyEdge2(post1,post2,0,delta,delta,0)
            else:
                gm.energyEdge1(post1,delta,0)
        elif post2!=-1:
            gm.energyEdge1(post2,delta,0)
    
    if d1==d2 and pre1>=0 and pre2>=0:
        assert(d1!=alpha and d1!=occ)
        delta=smooth_penalty(p,q,d1)
        gm.energyEdge2(pre1,pre2,0,delta,delta,0)

    if d1!=d2 and pre1>=0 and in_bounds((q[0],q[1]+d1)):
        delta=smooth_penalty(p,q,d1)
        gm.energyEdge1(pre1,delta,0)
    
    if d1!=d2 and pre2>=0 and in_bounds((p[0],p[1]+d2)):
        delta=smooth_penalty(p,q,d2)
        gm.energyEdge1(pre2,delta,0)

def buildUnique(gm,p,alpha):
    pre=vars_pre[p]
    post=vars_post[p]
    if pre<0:
        return
    if post!=-2:
        gm.maxweight(pre,post,occ)
    d=disparity_map[p]
    assert(d!=occ)
    newP=(p[0],p[1]+d-alpha)
    if in_bounds(newP):
        newPost=vars_post[newP]
        assert(newPost>=0)
        gm.maxweight(pre,newPost,occ)

def update_disparity(gm,alpha):
    global disparity_map, vars_pre, vars_post
    for i,pre in np.ndenumerate(vars_pre):
        if pre>=0 and gm.get_var(pre)==1:
            disparity_map[i]=occ
    for i,post in np.ndenumerate(vars_post):
        if post>=0 and gm.get_var(post)==1:
            disparity_map[i]=alpha


def expansionMove(alpha):
    
    global currEnergy,K,lambda1,lambda2,denom
    nb=L.shape[0]*L.shape[1]
    print("Creating graph with", nb*2, "nodes and", nb*12, "edges")
    gm=graphmaker.Graph(nb*2, nb*12)

    for p,_ in np.ndenumerate(iterator):
       buildNodes(gm,p,alpha)

    print("PRE:", vars_pre)
    print("POST:", vars_post)
    print("DISP:", disparity_map)
    print("Params:", K, lambda1, lambda2, denom)



    for p, _ in np.ndenumerate(iterator):
        for n in nbs:
            p2=(p[0]+n[0],p[1]+n[1])
            if in_bounds(p2):
                buildSmooth(gm,p,p2,alpha)
    
    for p,_ in np.ndenumerate(iterator):
        buildUnique(gm,p,alpha)

    oldEnergy=currEnergy
    currEnergy=gm.minimize()
    if currEnergy<oldEnergy:
        update_disparity(gm,alpha)
        return True
    currEnergy=oldEnergy
    return False


In [1]:
def saveImg(path):
	im = np.zeros(shape=(L.shape[0],L.shape[1], 3), dtype=np.uint8)
	im[:, :, 0] = 0
	im[:, :, 1] = 0
	im[:, :, 2] = 255

	dispSize = dispMax - dispMin
	idxs = np.argwhere(disparity_map != occ)
	c = 255 * (disparity_map[idxs] - dispMin) / dispSize if dispSize != 0 else 255
	im[idxs[:, 0], idxs[:, 1], :] = np.stack((c, c, c), axis=1)
	cv2.imwrite(path, im)
	print("Done!")

In [8]:
%%time
setParams()
tom=tomasi.Tomasi()
print("Params now:", K, lambda1, lambda2, denom)
currEnergy = findEnergy()
print("ENERGY: ", currEnergy)
done=np.full(dispSize,False)
nDone=dispSize

for i in range(maxIter):
	if nDone<=0:
		break
	
	perm=np.random.permutation(dispSize)
	
	for j in range(dispSize):
		d=perm[j]
		if done[d]:
			continue
		print("EXPMOVE:",d)
		if expansionMove(dispMin+d):
			done = np.full(dispSize, False)
			nDone=dispSize
		done[d]=True
		nDone=-1

	print("ENERGY :",currEnergy)
saveImg("/kaggle/working/dispMap1.png")


K obtained: 56.70314472853536
Params now: 851 510 170 15
ENERGY: 0
EXPMOVE: 3
Creating graph with 55296 nodes and 331776 edges
PRE: [[-2 -2 -2 ... -2 -2 -2]
 [-2 -2 -2 ... -2 -2 -2]
 [-2 -2 -2 ... -2 -2 -2]
 ...
 [-2 -2 -2 ... -2 -2 -2]
 [-2 -2 -2 ... -2 -2 -2]
 [-2 -2 -2 ... -2 -2 -2]]
POST: [[   -2    -2    -2 ...   184   185   186]
 [   -2    -2    -2 ...   371   372   373]
 [   -2    -2    -2 ...   558   559   560]
 ...
 [   -2    -2    -2 ... 26551 26552 26553]
 [   -2    -2    -2 ... 26738 26739 26740]
 [   -2    -2    -2 ... 26925 26926 26927]]
DISP: [[2147483647 2147483647 2147483647 ... 2147483647 2147483647 2147483647]
 [2147483647 2147483647 2147483647 ... 2147483647 2147483647 2147483647]
 [2147483647 2147483647 2147483647 ... 2147483647 2147483647 2147483647]
 ...
 [2147483647 2147483647 2147483647 ... 2147483647 2147483647 2147483647]
 [2147483647 2147483647 2147483647 ... 2147483647 2147483647 2147483647]
 [2147483647 2147483647 2147483647 ... 2147483647 2147483647 21474